In [1]:
#!/usr/bin/env python3
"""
Read SO-CI output from Molpro.
Assign omegas, including parity. 
Also show composition of a level, or distribution of a term.
C2v symmetry (4 irreps) is assumed. 
Includes dipole moments
This version uses both elimination and trig strategies
KKI 6/6/2023
"""
import re, sys, copy, glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, '../atomic_SOC')
import molpro_subs as mpr
import chem_subs as chem

pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [2]:
fsoci = 'ac5z_hybB_r2p2444_lz.pro'
#fsoci = '../UMemphis/mgcl-equil-karl-block.pro'
#fsoci = '../UMemphis/ch.pro'
#fsoci = '../UMemphis/mgcl_small4.pro'
print('Will read SO-CI info from file: {:s}'.format(fsoci))

Will read SO-CI info from file: ac5z_hybB_r2p2444_lz.pro


In [3]:
# Read CASSCF
PG = mpr.read_point_group(fsoci)
print('Point group is', PG)
crd, lineno_crd = mpr.read_coordinates(fsoci, linenum=True)
if isinstance(lineno_crd, list):
    # take last geometry
    crd = crd[-1]
    lineno_crd = lineno_crd[-1]
# get diatomic bond length
G = chem.Geometry(crd, intype='DataFrame', units='bohr')
G.toAngstrom()
R = np.round(G.distance(0, 1), 6)  # round the bond length to 6 digits
print('Bond length = {:.4f}'.format(R))
caslist, lineno_cas = mpr.readMULTI(fsoci, PG=PG, linenum=True)
CAS = caslist[-1]   # assume the last CASSCF to be the relevant one

oldcas = CAS.results.copy()
for Spin in sorted(set(oldcas.Spin)):
    nspin = len(oldcas[oldcas.Spin == Spin])
    print('{:d} {:s}s'.format(nspin, Spin))
print('Active space = {:d}/{:d}'.format(CAS.nactel(), CAS.nactorb()))
CAS.results = mpr.relabel_CAS_by_energy(CAS.results)
# Note any changes in CAS labeling
diflbl = (oldcas.Label != CAS.results.Label).values
if diflbl.any():
    dflbl = oldcas[['Label', 'Term']].copy()
    dflbl['NewLabel'] = CAS.results.Label
    print('Some CAS labels changed:')
    display(dflbl)
# check for dynamical weights
rx_dynw = re.compile('[~!]*dynw,(\d+)')
dynw = 0
with open(fsoci, 'r') as F:
    for line in F:
        m = rx_dynw.search(line)
        if m:
            dynw = int(m.group(1))
            print('Dynamical weighting with dynw = {:d}'.format(dynw))
if not dynw:
    print('Uniform weighting')

Point group is C2v
Bond length = 2.2444
35 Doublets
15 Quartets
Active space = 19/11
Dynamical weighting with dynw = 8
Dynamical weighting with dynw = 8


In [4]:
# count the terms that are included in the calculation
from collections import Counter
for spin, grp in CAS.results.groupby('Spin'):
    print(spin)
    print(Counter(grp.Term.tolist()))
#CAS.results

Doublet
Counter({'2Π': 12, '2Δ': 10, '2Σ+': 5, '2Φ': 4, '2Γ': 2, '2Σ-': 2})
Quartet
Counter({'4Π': 6, '4Δ': 4, '4Φ': 2, '4Σ-': 2, '4Σ+': 1})


In [5]:
casdf = CAS.results[['Irrep', 'Label', 'Energy', 'Term']].copy()
casdf['S'] = [chem.spinname(x)-1 for x in CAS.results.Spin]
casdf['Lz'] = np.round(np.sqrt(np.abs(CAS.results.LzLz)), 0).astype(int)
spin = sorted(set(casdf.S))
irreps = sorted(set(casdf.Irrep))
lzvals = sorted(set(casdf.Lz))

In [6]:
casdf['ecm'] = np.round((casdf.Energy - casdf.Energy.min()) * chem.AU2CM, 0)
casdf.sort_values('Energy')

,Irrep,Label,Energy,Term,S,Lz,ecm
0,1,1.1,-119.041268,2Δ,1,2,0.0
27,4,1.4,-119.041268,2Δ,1,2,0.0
1,1,2.1,-119.035373,2Σ+,1,0,1294.0
19,3,1.3,-119.032982,2Π,1,1,1819.0
11,2,1.2,-119.032982,2Π,1,1,1819.0
35,1,1.1,-118.992903,4Δ,3,2,10615.0
46,4,1.4,-118.992903,4Δ,3,2,10615.0
38,2,1.2,-118.990509,4Π,3,1,11140.0
42,3,1.3,-118.990509,4Π,3,1,11140.0
47,4,2.4,-118.985873,4Σ-,3,0,12158.0


In [7]:
# check for proper pairing of degenerate states
broken = False
for grp, dfg in casdf.groupby(['S', 'Lz']):
    if grp[1] == 0:
        # ignore Sigma states
        continue
    # number in each irrep should be equal
    grpi = dfg.groupby('Irrep')
    lens = grpi.size().values
    if lens[0] != lens[1]:
        print('Broken pair somewhere')
        display(dfg.sort_values('Energy'))
        broken = True
if not broken:
    print('All CASSCF state pairs are closed')

All CASSCF state pairs are closed


In [8]:
# check for non-integer Lz values
cruft = casdf.Lz - np.round(casdf.Lz, 0)
if cruft.any():
    print('*** There are non-integer values of Lz')
else:
    print('Lz values look OK')

Lz values look OK


In [9]:
# order the CASSCF states by energy within each (S, irrep) group
newdf = pd.DataFrame(columns=casdf.columns)
for lbl, dg in casdf.groupby(['S', 'Irrep']):
    dt = dg.copy().sort_values('Energy').reset_index(drop=True)
    #display(dt)
    newdf = newdf.append(dt)

In [10]:
def assign_trig(Lzs, irreps):
    # Add the sin/cos descriptor based upon Lz and irrep
    trig = []
    for L, irr in zip(Lzs, irreps):
        if L == 0:
            trig.append('1')
        elif L%2 == 1:
            # odd L
            if irr == 2:
                trig.append('cos')
            elif irr == 3:
                trig.append('sin')
            else:
                trig.append('???')
        else:
            # even L
            if irr == 1:
                trig.append('cos')
            elif irr == 4:
                trig.append('sin')
            else:
                trig.append('???')
    return trig

In [11]:
# read MRCI
cilist, lineno_ci = mpr.readMRCI(fsoci, linenum=True)   # probably many
for m in cilist:
    m.transfer_lz(CAS.results)
mrci = [mpr.MRCIstate(row) for m in cilist for (irow, row) in m.results.iterrows()]
dfci = mpr.combineMRCI(cilist)
# Add the trig column
dfci['trig'] = assign_trig(dfci.Lz, dfci.Irrep)

In [12]:
dfci['ecm'] = np.round( (dfci.Edav - dfci.Edav.min()) * chem.AU2CM, 1)
dfci.sort_values('Edav')

,Group,Spin,Irrep,Label,Energy,Edav,Ncore,dipX,dipY,dipZ,Eref,Dipole,Ref,C0,Configs,Lz,Term,trig,ecm
29,9,Doublet,4,1.4,-119.713500,-119.784065,0,0.0,0.0,-0.588169,-119.041268,0.588169,1.4,0.951341,"{'2222202222a': 0.7818616, '22a22a2222b': 0.03...",2.0,2Δ,sin,0.0
5,2,Doublet,1,1.1,-119.713500,-119.784064,0,0.0,0.0,-0.588175,-119.041268,0.588175,1.1,0.951341,"{'222aab22222': 0.2129926, '222aba22222': -0.0...",2.0,2Δ,cos,0.1
0,1,Doublet,1,2.1,-119.711771,-119.783698,0,0.0,0.0,-0.686412,-119.035373,0.686412,2.1,0.950797,"{'2222a022222': 0.7224706, '22220a22222': 0.10...",0.0,2Σ+,1,80.5
19,6,Doublet,3,1.3,-119.706564,-119.777318,0,0.0,0.0,-0.652485,-119.032982,0.652485,1.3,0.951300,"{'22a22a222b2': 0.1000028, '2222aa222b2': -0.0...",1.0,2Π,sin,1480.8
11,4,Doublet,2,1.2,-119.706564,-119.777318,0,0.0,0.0,-0.652485,-119.032982,0.652485,1.2,0.951300,"{'22a22a2b222': 0.1000028, '2222aa2b222': -0.0...",1.0,2Π,cos,1480.8
48,18,Quartet,4,1.4,-119.660270,-119.730043,0,0.0,0.0,-0.161304,-118.992903,0.161304,1.4,0.951564,"{'2222aa2222a': 0.9437512, '22a22a2222a': -0.1...",2.0,4Δ,sin,11856.4
36,12,Quartet,1,1.1,-119.660270,-119.730043,0,0.0,0.0,-0.161305,-118.992903,0.161305,1.1,0.951564,"{'222aaa22222': 0.9437512, '22aa2a22222': 0.11...",2.0,4Δ,cos,11856.4
1,1,Doublet,1,3.1,-119.656274,-119.727746,0,0.0,0.0,-0.014132,-118.985396,0.014132,3.1,0.950718,"{'2222a022222': -0.0927829, '22220a22222': 0.6...",0.0,2Σ+,1,12360.5
42,15,Quartet,3,1.3,-119.657255,-119.727039,0,0.0,0.0,-0.096289,-118.990509,0.096289,1.3,0.951552,"{'2222aa222a2': 0.8773686, '22a22a222a2': -0.2...",1.0,4Π,sin,12515.7
38,13,Quartet,2,1.2,-119.657255,-119.727039,0,0.0,0.0,-0.096289,-118.990509,0.096289,1.2,0.951552,"{'2222aa2a222': 0.8773686, '22a22a2a222': -0.2...",1.0,4Π,cos,12515.7


In [13]:
# check for erroneously repeated CI roots
tol = 1.e-6
for irrep, grp in dfci.sort_values('Energy').groupby(['Spin', 'Irrep']):
    e = grp.Energy.values
    de = e[1:] - e[:-1]
    smal = np.abs(de) < tol
    for i, s in enumerate(smal):
        if s:
            print('Warning: closely repeated root in MRCI')
            display(grp.iloc[[i,i+1]])
print('Checking for discrepancies between reference energy and CASSCF energy')
dfcheck = dfci[['Spin', 'Irrep', 'Label', 'Edav', 'Eref']].copy()
ecas = []
for i, row in dfcheck.iterrows():
    ecas.append(CAS.results[(CAS.results.Label == row.Label) & (CAS.results.Spin == row.Spin)].Energy.values[0])
dfcheck['CAS'] = ecas
dfcheck['diff'] = np.round(dfcheck.CAS - dfcheck.Eref, 6)
dfbad = dfcheck[np.abs(dfcheck['diff']) > 0.1].sort_values(['Spin', 'Irrep', 'Label'])
if len(dfbad):
    display(dfbad)
else:
    print('\t--looks good')

Checking for discrepancies between reference energy and CASSCF energy
	--looks good


In [14]:
# read SO-CI
SOCI = mpr.fullmatSOCI(fsoci, hybrid=True)
dfterms = SOCI.average_terms()

Computational group = C2v
CASSCF states:
    35 Doublet
    15 Quartet
Replacing MRCI+Q energies by HLSDIAG values
*** Warning: "130 SO states but only 84 unique basis states" in parse_SOmatrix()


In [15]:
# Assign leading terms
lTerm = []  # list of str
for ilead in np.argmax(SOCI.termwt, axis=0):
    lTerm.append(SOCI.dfterm.at[ilead, 'Term'])
dfso = SOCI.SOe.results.copy()
dfso['Leading'] = lTerm
# All term weights
twts = []   # list of dict
for iso in range(SOCI.dimen):
    twts.append({t: np.round(w, 4) for t, w in zip(SOCI.dfterm.Term.values, SOCI.termwt[:, iso])})
dfso['Termwts'] = twts
print('Term weights rounded to 1e-4')
dfso = dfso.rename(columns={'Erel': 'cm-1'})  # rename column Erel to cm-1
SOCI.dfso = dfso
dfso

Term weights rounded to 1e-4


,Nr,Irrep,E,Eshift,cm-1,Leading,Termwts
0,1,0,-119.860885,-4170.78,0.00,(1)2Δ,"{'(1)2Δ': 0.9829, '(1)2Σ+': 0.0, '(1)2Π': 0.0,..."
1,2,0,-119.860885,-4170.78,0.00,(1)2Δ,"{'(1)2Δ': 0.9829, '(1)2Σ+': 0.0, '(1)2Π': 0.0,..."
2,3,0,-119.854930,-2863.76,1307.02,(1)2Σ+,"{'(1)2Δ': 0.0, '(1)2Σ+': 0.6869, '(1)2Π': 0.29..."
3,4,0,-119.854930,-2863.76,1307.02,(1)2Σ+,"{'(1)2Δ': 0.0, '(1)2Σ+': 0.6869, '(1)2Π': 0.29..."
4,5,0,-119.854576,-2785.94,1384.84,(1)2Π,"{'(1)2Δ': 0.2754, '(1)2Σ+': 0.0, '(1)2Π': 0.70..."
5,6,0,-119.854576,-2785.94,1384.84,(1)2Π,"{'(1)2Δ': 0.2754, '(1)2Σ+': 0.0, '(1)2Π': 0.70..."
6,7,0,-119.816552,5559.29,9730.07,(1)2Π,"{'(1)2Δ': 0.0, '(1)2Σ+': 0.2431, '(1)2Π': 0.59..."
7,8,0,-119.816552,5559.29,9730.07,(1)2Π,"{'(1)2Δ': 0.0, '(1)2Σ+': 0.2431, '(1)2Π': 0.59..."
8,10,0,-119.816169,5643.38,9814.16,(1)2Δ,"{'(1)2Δ': 0.6697, '(1)2Σ+': 0.0, '(1)2Π': 0.28..."
9,9,0,-119.816169,5643.38,9814.16,(1)2Δ,"{'(1)2Δ': 0.6697, '(1)2Σ+': 0.0, '(1)2Π': 0.28..."


In [16]:
# Create a DataFrame to describe the basis states
cilbl = []
irrep = []
Ss = []
Szs = []
for bas in SOCI.basis:
    cilbl.append(bas[0])
    i, irp = bas[0].split('.')
    irrep.append(irp)
    Ss.append(bas[1])
    Szs.append(bas[2])
dfbas = pd.DataFrame({'cilbl': cilbl, 'irrep': irrep, 'S': Ss, 'Sz': Szs})
# Add values of |Lz|
Lz = [dfci.loc[ici, 'Lz'] for ici in SOCI.sob_ici]
dfbas['|Lz|'] = Lz

In [17]:
# Pair the basis states that are sin/cos siblings
sib = [-1] * SOCI.dimen
trig = [dfci.at[ici, 'trig'] for ici in SOCI.sob_ici]
dfbas['trig'] = trig
baspairs = []
for iterm, trow in SOCI.dfterm.iterrows():
    ibass = SOCI.term_iso[iterm]
    for ibas in ibass:
        if sib[ibas] > -1:
            # already assigned
            continue
        for jbas in ibass:
            if (jbas != ibas) and (dfbas.at[ibas, 'S'] == dfbas.at[jbas, 'S']) and \
                            (dfbas.at[ibas, 'Sz'] == dfbas.at[jbas, 'Sz']):
                sib[ibas] = jbas
                sib[jbas] = ibas
                baspairs.append([ibas, jbas])
dfbas['sib'] = sib
print('Basis states')
dfbas

Basis states


,cilbl,irrep,S,Sz,|Lz|,trig,sib
0,2.1,1,0.5,0.5,0.0,1,-1
1,3.1,1,0.5,0.5,0.0,1,-1
2,6.1,1,0.5,0.5,0.0,1,-1
3,8.1,1,0.5,0.5,0.0,1,-1
4,11.1,1,0.5,0.5,0.0,1,-1
5,2.1,1,0.5,-0.5,0.0,1,-1
6,3.1,1,0.5,-0.5,0.0,1,-1
7,6.1,1,0.5,-0.5,0.0,1,-1
8,8.1,1,0.5,-0.5,0.0,1,-1
9,11.1,1,0.5,-0.5,0.0,1,-1


In [18]:
# Begin assignments
omegavals = mpr.omega_counts(mrci, silent=True)
print('Target omega counts:', omegavals)
omeganeed = omegavals.copy()
# add column of possible Omegas to dfso
omposs = [set(omegavals.keys()) for i in range(SOCI.dimen)]
dfso['omposs'] = omposs
dfso[chem.OMEGA] = None
wthresh = 0.01  # ignore weights smaller than 'wthresh'
print('Weight threshold = {:.4f}'.format(wthresh))

Target omega counts: {0.5: 48, 1.5: 40, 2.5: 26, 3.5: 12, 4.5: 4}
Weight threshold = 0.0100


In [19]:
def assign_state(iso, omega, omeganeed, dfso):
    # Assign the state 'iso' to Ω = 'omega'
    #   'omeganeed' is the dict of needed values (modified)
    #   'dfso' is a DataFrame of SO states (modified)
    # Return success flag and a message

    # Is this state already assigned?
    if dfso.at[iso, 'Ω'] is not None:
        # already assigned
        ok = (omega == dfso.at[iso, 'Ω'])  # failure if this is a different value
        return ok, f'conflict of {omega} with earlier assignment {dfso.at[iso, "Ω"]}'
    # Is 'omega' among the remaining possiblities for this state?
    if not omega in dfso.at[iso, 'omposs']:
        return False, 'already excluded for this state'
    # Is 'omega' still needed?
    if omeganeed[omega] < 1:
        return False, f'exceeds target count for Ω = {omega}'
    # update 'omeganeed' and 'dfso'
    omeganeed[omega] -= 1
    dfso.at[iso, 'omposs'] = set([omega])
    dfso.at[iso, 'Ω'] = omega
    return True, 'new assignment'

def check_fulfillment(omeganeed, dfso, verbose=True):
    # If any omega counts have been reached, delete
    #   the corresponding omegas from possible future
    #   assignments
    # Return the number of affected states
    totchange = 0
    for om, n in omeganeed.items():
        nchange = 0
        if n > 1:
            # count is not satisfied 
            continue
        # count is satisfied for Omega = om
        for iso, row in dfso.iterrows():
            oset = row['omposs']
            if (len(oset) > 1) and (om in oset):
                # it is listed; remove it
                oset.remove(om)
                dfso.at[iso, 'omposs'] = oset
                nchange += 1
        if nchange and verbose:
            print(f'Omega count satisfied for Ω = {om}: possibilities updated for {nchange} states')
        totchange += nchange
    return totchange

def assign_singletons(dfso, omeganeed):
    # If there are unassigned states with only one possible
    #   value of Omega, assign them
    # Return the number of states so assigned
    inotassigned = dfso[dfso.Ω.isnull()].index
    nnew = 0
    for iso in inotassigned:
        omposs = list(dfso.at[iso, 'omposs'])
        nposs = len(omposs)
        if nposs == 1:
            # assign this state
            assign_state(iso, omposs[0], omeganeed, dfso)
            nnew += 1
    return nnew

In [20]:
# Sigma states are the easiest, so start with them
eigvec = SOCI.SOvec.copy()  # Eigenvectors from Molpro; will it work with my vectors?
eigvec = SOCI.vec  # eigenvectors from re-diagonalization
print('Starting with Lz = 0 basis states')
nnew = 0
for ibas, row in dfbas.iterrows():
    if row['|Lz|'] != 0:
        continue
    # Lz = 0
    om = abs(row.Sz)  # Ω = 0 + Sz
    #print(ibas, row['|Lz|'], om)
    # Assign the SO states to which this BS contributes sufficient weight
    wts = np.absolute(eigvec[ibas, :])
    idx = np.argwhere(wts > wthresh).flatten()
    for iso in idx:
        ok, msg = assign_state(iso, om, omeganeed, dfso)
        if ok and (msg == 'new assignment'):
            print(f'\tassigned state #{iso} with Ω = {om}')
            nnew += 1
        if not ok:
            print(f'*** Assignment failed:  ibas = {ibas}, iso = {iso}, omega = {om}: {msg}')
print(f'\t---{nnew} states assigned---')
print('Omegas still needed: ', omeganeed)

Starting with Lz = 0 basis states
	assigned state #3 with Ω = 0.5
	assigned state #7 with Ω = 0.5
	assigned state #17 with Ω = 0.5
	assigned state #22 with Ω = 0.5
	assigned state #23 with Ω = 0.5
	assigned state #29 with Ω = 0.5
	assigned state #31 with Ω = 0.5
	assigned state #35 with Ω = 0.5
	assigned state #48 with Ω = 0.5
	assigned state #55 with Ω = 0.5
	assigned state #58 with Ω = 0.5
	assigned state #63 with Ω = 0.5
	assigned state #71 with Ω = 0.5
	assigned state #73 with Ω = 0.5
	assigned state #76 with Ω = 0.5
	assigned state #78 with Ω = 0.5
	assigned state #87 with Ω = 0.5
	assigned state #89 with Ω = 0.5
	assigned state #97 with Ω = 0.5
	assigned state #100 with Ω = 0.5
	assigned state #118 with Ω = 0.5
	assigned state #123 with Ω = 0.5
	assigned state #28 with Ω = 0.5
	assigned state #72 with Ω = 0.5
	assigned state #79 with Ω = 0.5
	assigned state #128 with Ω = 0.5
	assigned state #98 with Ω = 0.5
	assigned state #112 with Ω = 0.5
	assigned state #113 with Ω = 0.5
	assi

In [21]:
npruned = check_fulfillment(omeganeed, dfso)
while npruned:
    # assign any states with only one remaining possilibity
    npruned = 0
    nnew = assign_singletons(dfso, omeganeed)
    if nnew:
        npruned = check_fulfillment(omeganeed, dfso)
print('Assignments still needed:', omeganeed)

Omega count satisfied for Ω = 0.5: possibilities updated for 42 states
Omega count satisfied for Ω = 1.5: possibilities updated for 42 states
Assignments still needed: {0.5: 0, 1.5: 0, 2.5: 26, 3.5: 12, 4.5: 4}


In [22]:
# The trig method does not include basis states with Lz = 0
print('Using trig method to assign more states')
print('Everything below is numbered from zero, but in Molpro is numbered from 1')
Omegas = [None] * SOCI.dimen
for iso in range(SOCI.dimen):
    print(f'Eigenvector #{iso}')
    vec = eigvec[:, iso]
    #print(np.round(vec, 6))
    omset = set()  # set of found Omega values--should end up with only one element
    usedBS = []  # list of basis states considered
    # Basis states are considered in pairs
    for baspair in baspairs:
        # 'a' and 'b' are coeffs of cos() and sin()
        [i, j] = baspair
        usedBS.extend(baspair)
        aLz = dfbas.at[i, '|Lz|']  # same for basis-state[i] and BS[j]
        Sz = dfbas.at[i, 'Sz']  # same for i and j
        if dfbas.at[i, 'trig'] == 'cos':
            # cos, sin ordering
            [a, b] = [vec[i], vec[j]]
        else:
            # sin, cos ordering
            [a, b] = [vec[j], vec[i]]
        #print('coeffs:', vec[i], vec[j])
        alen = np.absolute(a)
        blen = np.absolute(b)
        magn = (alen + blen) / 2
        delta = alen - blen
        if abs(delta) > wthresh:
            print('\tBSs:', baspair, f'with magn = {magn:.3f}')
            print('** large difference in magnitudes = {:.8f}'.format(delta))
        if min(alen, blen) < wthresh:
            # ignore small coefficients
            continue
        # divide by a to get standard form
        b /= a
        a = 1
        #print('a, b  :', a, b)
        dev = abs(np.imag(b)) - 1
        if abs(dev) > wthresh:
            print('\tBSs:', baspair, f'with magn = {magn:.3f}')
            print('** imag(b) deviates from unity by: {:.8f}'.format(dev))
        if np.imag(b) > wthresh:
            # a positive combination
            Lz = aLz
        elif np.imag(b) < -wthresh:
            # a negative combination
            Lz = -aLz
        Omz = Lz + Sz
        print('\tBSs:', baspair, f'with magn = {magn:.3f}')
        print(f'\t\tLz = {Lz}, Sz = {Sz}, Omz = {Omz}')
        omset = omset.union([abs(Omz)])
        
    # finish up
    Omegas[iso] = omset
    if len(omset) == 1:
        Omegas[iso] = omset
    else:
        print('\t*** Multiple Omega values:', omset)

Using trig method to assign more states
Everything below is numbered from zero, but in Molpro is numbered from 1
Eigenvector #0
	BSs: [58, 10] with magn = 0.701
		Lz = 2.0, Sz = 0.5, Omz = 2.5
	BSs: [98, 82] with magn = 0.015
		Lz = 1.0, Sz = 1.5, Omz = 2.5
	BSs: [59, 11] with magn = 0.022
		Lz = 2.0, Sz = 0.5, Omz = 2.5
	BSs: [96, 112] with magn = 0.048
		Lz = 3.0, Sz = -0.5, Omz = 2.5
	BSs: [83, 99] with magn = 0.051
		Lz = 1.0, Sz = 1.5, Omz = 2.5
	BSs: [36, 52] with magn = 0.022
		Lz = 3.0, Sz = -0.5, Omz = 2.5
	BSs: [84, 100] with magn = 0.037
		Lz = 1.0, Sz = 1.5, Omz = 2.5
	BSs: [37, 53] with magn = 0.030
		Lz = 3.0, Sz = -0.5, Omz = 2.5
Eigenvector #1
	BSs: [63, 15] with magn = 0.701
		Lz = -2.0, Sz = -0.5, Omz = -2.5
	BSs: [107, 91] with magn = 0.015
		Lz = -1.0, Sz = -1.5, Omz = -2.5
	BSs: [64, 16] with magn = 0.022
		Lz = -2.0, Sz = -0.5, Omz = -2.5
	BSs: [95, 111] with magn = 0.048
		Lz = -3.0, Sz = 0.5, Omz = -2.5
	BSs: [92, 108] with magn = 0.051
		Lz = -1.0, Sz = -1.5, O

	BSs: [28, 44] with magn = 0.011
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [98, 82] with magn = 0.020
		Lz = -1.0, Sz = 1.5, Omz = 0.5
	BSs: [104, 88] with magn = 0.017
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [29, 45] with magn = 0.047
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [30, 46] with magn = 0.101
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [83, 99] with magn = 0.039
		Lz = -1.0, Sz = 1.5, Omz = 0.5
	BSs: [86, 102] with magn = 0.018
		Lz = -1.0, Sz = 0.5, Omz = -0.5
	BSs: [89, 105] with magn = 0.228
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [75, 123] with magn = 0.027
		Lz = 2.0, Sz = 1.5, Omz = 3.5
	BSs: [81, 129] with magn = 0.351
		Lz = -2.0, Sz = -1.5, Omz = -3.5
	BSs: [25, 41] with magn = 0.022
		Lz = -1.0, Sz = 0.5, Omz = -0.5
	BSs: [31, 47] with magn = 0.285
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [84, 100] with magn = 0.080
		Lz = -1.0, Sz = 1.5, Omz = 0.5
	BSs: [87, 103] with magn = 0.020
		Lz = -1.0, Sz = 0.5, Omz = -0.5
	BSs: [90, 106] with magn = 0.261
		Lz = 1.0, Sz = -0.5, Omz = 0.

	BSs: [104, 88] with magn = 0.016
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [29, 45] with magn = 0.045
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [30, 46] with magn = 0.013
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [83, 99] with magn = 0.036
		Lz = -1.0, Sz = 1.5, Omz = 0.5
	BSs: [81, 129] with magn = 0.043
		Lz = -2.0, Sz = -1.5, Omz = -3.5
	BSs: [31, 47] with magn = 0.170
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [84, 100] with magn = 0.289
		Lz = -1.0, Sz = 1.5, Omz = 0.5
	BSs: [90, 106] with magn = 0.505
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [32, 48] with magn = 0.098
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [33, 49] with magn = 0.013
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	*** Multiple Omega values: {0.5, 3.5}
Eigenvector #99
	BSs: [101, 85] with magn = 0.016
		Lz = -1.0, Sz = 0.5, Omz = -0.5
	BSs: [23, 39] with magn = 0.045
		Lz = -1.0, Sz = 0.5, Omz = -0.5
	BSs: [24, 40] with magn = 0.013
		Lz = -1.0, Sz = 0.5, Omz = -0.5
	BSs: [92, 108] with magn = 0.036
		Lz = 1.0, Sz = -1.5, Omz = -0.5
	BSs: [75

In [23]:
# Assign states where trig analysis gave a single Omega
print('Assigning any states with clean trig analysis')
nnew = 0
for iso, omset in enumerate(Omegas):
    if len(omset) == 1:
        # a clean value
        om = list(omset)[0]
        ok, msg = assign_state(iso, om, omeganeed, dfso)
        if ok and (msg == 'new assignment'):
            print(f'\tassigned state #{iso} with Ω = {om}')
            nnew += 1
        if not ok:
            print(f'*** Assignment failed:  ibas = {ibas}, iso = {iso}, omega = {om}: {msg}')
print(f'\t---{nnew} states assigned---')
print('Omegas still needed: ', omeganeed)

Assigning any states with clean trig analysis
	assigned state #0 with Ω = 2.5
	assigned state #1 with Ω = 2.5
	assigned state #18 with Ω = 4.5
	assigned state #19 with Ω = 4.5
	assigned state #104 with Ω = 4.5
	assigned state #105 with Ω = 4.5
	---6 states assigned---
Omegas still needed:  {0.5: 0, 1.5: 0, 2.5: 24, 3.5: 12, 4.5: 0}


In [24]:
# Assign states where trig analysis gave multiple Omegas
print('Assigning states with mixed trig analysis')
nnew = 0
for iso, row in dfso.iterrows():
    if len(Omegas[iso]) < 2:
        continue
    oms = row.omposs.intersection(Omegas[iso])  # intersection of omposs with trig results
    if len(oms) == 1:
        # a possible new assignment
        om = list(oms)[0]
        ok, msg = assign_state(iso, om, omeganeed, dfso)
        if ok and (msg == 'new assignment'):
            print(f'\tassigned state #{iso} with Ω = {om}')
            nnew += 1
        if not ok:
            print(f'*** Assignment failed:  ibas = {ibas}, iso = {iso}, omega = {om}: {msg}')
print(f'\t---{nnew} states assigned---')
print('Omegas still needed: ', omeganeed)

Assigning states with mixed trig analysis
	assigned state #10 with Ω = 3.5
	assigned state #11 with Ω = 3.5
	assigned state #12 with Ω = 2.5
	assigned state #13 with Ω = 2.5
	assigned state #20 with Ω = 2.5
	assigned state #21 with Ω = 2.5
	assigned state #26 with Ω = 3.5
	assigned state #27 with Ω = 3.5
	assigned state #38 with Ω = 2.5
	assigned state #39 with Ω = 2.5
	assigned state #40 with Ω = 3.5
	assigned state #41 with Ω = 3.5
	assigned state #42 with Ω = 2.5
	assigned state #43 with Ω = 2.5
	assigned state #46 with Ω = 2.5
	assigned state #47 with Ω = 2.5
	assigned state #56 with Ω = 2.5
	assigned state #57 with Ω = 2.5
	assigned state #64 with Ω = 3.5
	assigned state #65 with Ω = 3.5
	assigned state #66 with Ω = 2.5
	assigned state #67 with Ω = 2.5
	assigned state #82 with Ω = 2.5
	assigned state #83 with Ω = 2.5
	assigned state #90 with Ω = 2.5
	assigned state #91 with Ω = 2.5
	assigned state #106 with Ω = 3.5
	assigned state #107 with Ω = 3.5
	assigned state #108 with Ω = 2.

In [25]:
npruned = check_fulfillment(omeganeed, dfso)
while npruned:
    # assign any states with only one remaining possilibity
    npruned = 0
    nnew = assign_singletons(dfso, omeganeed)
    if nnew:
        npruned = check_fulfillment(omeganeed, dfso)
nneed = sum(omeganeed.values())
if nneed:
    print('Assignments still needed:', omeganeed)
else:
    print('All states assigned!')

All states assigned!


In [26]:
# Add term symbol labels and Omega labels
tsymb = []
olbl = []
for trm, om in zip(dfso.Leading, dfso.Ω):
    hom = chem.halves(om)
    olbl.append(hom)
    tsymb.append('_'.join([trm, hom]))
dfso['Tsymb'] = tsymb
dfso['Olbl'] = chem.enumerative_prefix(olbl, always=True)
SOCI.average_SO_levels(be_same=['Ω'], to_avg=['E', 'Eshift', 'cm-1'])
dflevel = SOCI.dflevel.reset_index(drop=True)
print(f'Assignments for {fsoci}')
dflevel[['Nr', 'E', 'Eshift', 'cm-1', 'Leading', 'Ω', 'g', 'Tsymb', 'Olbl']]

Assignments for ac5z_hybB_r2p2444_lz.pro


,Nr,E,Eshift,cm-1,Leading,Ω,g,Tsymb,Olbl
0,"[1, 2]",-119.860885,-4170.78,0.00,(1)2Δ,2.5,2,(1)2Δ_5/2,(1)5/2
1,"[3, 4]",-119.854930,-2863.76,1307.02,(1)2Σ+,0.5,2,(1)2Σ+_1/2,(1)1/2
2,"[5, 6]",-119.854576,-2785.94,1384.84,(1)2Π,1.5,2,(1)2Π_3/2,(1)3/2
3,"[7, 8]",-119.816552,5559.29,9730.07,(1)2Π,0.5,2,(1)2Π_1/2,(3)1/2
4,"[9, 10]",-119.816169,5643.38,9814.16,(1)2Δ,1.5,2,(1)2Δ_3/2,(4)3/2
5,"[11, 12]",-119.813865,6148.99,10319.77,(1)4Δ,3.5,2,(1)4Δ_7/2,(1)7/2
6,"[13, 14]",-119.812937,6352.71,10523.49,(1)4Δ,2.5,2,(1)4Δ_5/2,(3)5/2
7,"[15, 16]",-119.811231,6727.16,10897.95,(1)4Π,1.5,2,(1)4Π_3/2,(5)3/2
8,"[17, 18]",-119.809407,7127.53,11298.31,(1)4Π,0.5,2,(1)4Π_1/2,(6)1/2
9,"[19, 20]",-119.807160,7620.69,11791.47,4Φ,4.5,2,4Φ_9/2,(1)9/2


In [27]:
# Is this a hybrid calculation (prepared by build_hybrid_soci_input.ipynb)?
ccterms = []  # list of input CCSD(T) terms
rx_hyb = re.compile('HLSDIAG\(.+\s+!\s*.*(ccsd|anchored|shifted|input)')
is_hybrid = False
with open(fsoci, 'r', encoding='utf8') as F:
    for line in F:
        if rx_hyb.search(line):
            is_hybrid = True
            if 'input' in line:
                # an anchor term; extract its label
                words = line.split()
                if words[2] not in ccterms:
                    ccterms.append(words[2])
if is_hybrid:
    print('Hybrid SO-CI calculation')
else:
    print('Standard SO-CI')

Hybrid SO-CI calculation


In [28]:
# copy Energies to clipboard with term symbols as column headings
dfcp = dflevel[['Olbl', 'E']].copy().sort_values('Olbl').set_index('Olbl')
dfcp.rename(columns={'E': f'{R}'}, inplace=True)   # put bond length in that position for pasting to Excel
dfcp.T.to_clipboard()
if is_hybrid:
    print(f'Hybrid SO-CI energies for R={R} copied to clipboard, for pasting into Excel')
else:
    print(f'Standard SO-CI energies for R={R} copied to clipboard, for pasting into Excel')

Hybrid SO-CI energies for R=2.2444 copied to clipboard, for pasting into Excel


### Term composition of some levels

In [29]:
olabels = ['(1)1/2', '(1)3/2', '(1)5/2']
for olabel in olabels:
    ilev = dfso[SOCI.dfso.Olbl == olabel].index[0]
    #print('ilvel =', ilev)
    print('Composition of level #{:d}:  "{:s}" or "{:s}"'.format(ilev, dfso.loc[ilev].Olbl,
                                                             dfso.loc[ilev].Tsymb))
    dfci, dfterm = SOCI.composition_of_level(ilev, thr=1.e-6, normalize=True)
    wtcol = dfterm.columns[-1]
    display(dfterm.sort_values(wtcol, ascending=False))
    print('Sum of weights = {:.3f}'.format(dfterm[wtcol].sum()))
    ccsum = dfterm[dfterm.Term.isin(ccterms)][wtcol].sum()
    print('Sum of weights from CC terms = {:.3f}\n'.format(ccsum))

Composition of level #2:  "(1)1/2" or "(1)2Σ+_1/2"
Eso[2] = 1307.0 cm-1


,Term,dipZ,Edav,idx,ecm,wt_02
2,(1)2Π,-0.652485,-119.777318,"[11, 19]",1480.7,0.706705
0,(1)2Δ,-0.588172,-119.784064,"[29, 5]",0.0,0.275432
8,(1)4Σ-,-0.142579,-119.715854,[46],14970.4,0.005436
9,4Φ,-0.110737,-119.714348,"[41, 45]",15301.1,0.003421
12,(3)2Π,0.097561,-119.709794,"[13, 21]",16300.4,0.002956
26,(5)2Δ,-0.010376,-119.649384,"[9, 33]",29559.0,0.001437
13,(2)4Π,0.147435,-119.708154,"[39, 43]",16660.4,0.001115
20,(3)4Π,0.003020,-119.670024,"[40, 44]",25028.9,0.000850
16,(3)2Δ,0.332825,-119.703829,"[31, 7]",17609.7,0.000631
22,(4)2Δ,-0.160284,-119.657767,"[8, 32]",27719.0,0.000373


Sum of weights = 1.000
Sum of weights from CC terms = 0.982

Composition of level #4:  "(1)3/2" or "(1)2Π_3/2"
Eso[4] = 1384.8 cm-1


,Term,dipZ,Edav,idx,ecm,wt_04
0,(1)2Δ,-0.588172,-119.784064,"[29, 5]",0.0,0.669696
2,(1)2Π,-0.652485,-119.777318,"[11, 19]",1480.7,0.281520
9,4Φ,-0.110737,-119.714348,"[41, 45]",15301.1,0.019495
12,(3)2Π,0.097561,-119.709794,"[13, 21]",16300.4,0.011069
6,(2)2Δ,0.101961,-119.718361,"[30, 6]",14420.2,0.004233
18,(4)2Π,0.132814,-119.674217,"[14, 22]",24108.7,0.003696
7,(2)2Π,0.095556,-119.716350,"[12, 20]",14861.5,0.002808
24,(5)2Π,-0.066180,-119.653751,"[15, 23]",28600.6,0.002155
22,(4)2Δ,-0.160284,-119.657767,"[8, 32]",27719.0,0.001791
16,(3)2Δ,0.332825,-119.703829,"[31, 7]",17609.7,0.001640


Sum of weights = 1.000
Sum of weights from CC terms = 0.952

Composition of level #0:  "(1)5/2" or "(1)2Δ_5/2"
Eso[0] = 0.0 cm-1


,Term,dipZ,Edav,idx,ecm,wt_00
0,(1)2Δ,-0.588172,-119.784064,"[29, 5]",0.0,0.982950
13,(2)4Π,0.147435,-119.708154,"[39, 43]",16660.4,0.005210
9,4Φ,-0.110737,-119.714348,"[41, 45]",15301.1,0.004648
20,(3)4Π,0.003020,-119.670024,"[40, 44]",25028.9,0.002686
28,(2)2Φ,-0.046586,-119.648791,"[18, 26]",29689.1,0.001745
6,(2)2Δ,0.101961,-119.718361,"[30, 6]",14420.2,0.000975
14,(1)2Φ,0.069270,-119.706699,"[17, 25]",16979.7,0.000948
5,(1)4Π,-0.096289,-119.727039,"[42, 38]",12515.7,0.000426
26,(5)2Δ,-0.010376,-119.649384,"[9, 33]",29559.0,0.000187
16,(3)2Δ,0.332825,-119.703829,"[31, 7]",17609.7,0.000096


Sum of weights = 1.000
Sum of weights from CC terms = 0.983



### Distribution of a term among levels

In [30]:
term = '(1)2Σ+'
term = '(1)2Δ'
if is_hybrid:
    typ = 'hybrid'
else:
    typ = 'standard'
print('Distribution of term "{:s}" among {:s} levels'.format(term, typ))
print(f'R = {R}')
df = SOCI.level_contributions_from_term(term, thr=1.e-6, normalize=True,
                                       cols=['E', 'Eshift', 'Ω', 'Tsymb', 'Olbl'])
df['prod'] = np.round(df.Eshift * df[term], 1)
ebar = df['prod'].sum() / df[term].sum()
print('Weighted mean energy of {:s} = {:.1f} cm-1'.format(term, ebar))
#display(df.sort_values(term, ascending=False))
fmt = {term: '{:.4f}', 'prod': '{:.2f}', 'Eshift': '{:.2f}', 'Ω': '{:.1f}'}
display(df[df[term] > 0.0005].sort_values('E').style.format(fmt))
print('Total weight = {:.4f}'.format(df[term].sum()))

Distribution of term "(1)2Δ" among hybrid levels
R = 2.2444
Weighted mean energy of (1)2Δ = -3052.0 cm-1


,E,Eshift,Ω,Tsymb,Olbl,(1)2Δ,prod
0,-119.860885,-4170.78,2.5,(1)2Δ_5/2,(1)5/2,0.4915,-2049.80
2,-119.854930,-2863.76,0.5,(1)2Σ+_1/2,(1)1/2,0.1377,-394.40
4,-119.854576,-2785.94,1.5,(1)2Π_3/2,(1)3/2,0.3348,-932.90
10,-119.813865,6148.99,3.5,(1)4Δ_7/2,(1)7/2,0.0024,14.50
12,-119.812937,6352.71,2.5,(1)4Δ_5/2,(3)5/2,0.0086,54.70
16,-119.809407,7127.53,0.5,(1)4Π_1/2,(5)1/2,0.0015,10.50
18,-119.807160,7620.69,4.5,4Φ_9/2,(1)9/2,0.0011,8.10
19,-119.807160,7620.69,4.5,4Φ_9/2,(2)9/2,0.0008,5.80
21,-119.804187,8273.07,2.5,(2)2Δ_5/2,(6)5/2,0.0008,6.40
23,-119.803750,8369.05,0.5,(2)2Σ+_1/2,(8)1/2,0.0006,5.40


Total weight = 1.0000


### Contribution of CCSD(T) terms to all levels

In [31]:
# Get contributions of CC terms to all levels
# Loop over levels to get the normalization right
dfcc = SOCI.dfso.copy()
ccsums = []
for ilev in dfcc.index:
    dfci, dfterm = SOCI.composition_of_level(ilev, thr=1.e-6,
                                normalize=True, silent=True)
    wtcol = dfterm.columns[-1]
    ccsum = dfterm[dfterm.Term.isin(ccterms)][wtcol].sum()
    ccsums.append(ccsum)
dfcc['CCwt'] = ccsums
fmt = {chem.OMEGA: '{:.1f}', 'exc': '{:.2f}', 'CCwt': '{:.3f}'}
display(dfcc[dfcc.columns[2:]].style.format(fmt))
print('Total of CCwt column = {:.3f}'.format(dfcc.CCwt.sum()))
print('The CC terms are', ccterms)

,E,Eshift,cm-1,Leading,Termwts,omposs,Ω,Tsymb,Olbl,CCwt
0,-119.860885,-4170.780000,0.000000,(1)2Δ,"{'(1)2Δ': 0.9829, '(1)2Σ+': 0.0, '(1)2Π': 0.0, '(1)4Δ': 0.0001, '(2)2Σ+': 0.0, '(1)4Π': 0.0004, '(2)2Δ': 0.001, '(2)2Π': 0.0, '(1)4Σ-': 0.0, '4Φ': 0.0046, '(3)2Σ+': 0.0, '4Σ+': 0.0, '(3)2Π': 0.0, '(2)4Π': 0.0052, '(1)2Φ': 0.0009, '(1)2Σ-': 0.0, '(3)2Δ': 0.0001, '(2)4Δ': 0.0, '(4)2Π': 0.0, '(2)4Σ-': 0.0, '(3)4Π': 0.0027, '(2)2Σ-': 0.0, '(4)2Δ': 0.0, '(4)2Σ+': 0.0, '(5)2Π': 0.0, '2Γ': 0.0, '(5)2Δ': 0.0002, '(5)2Σ+': 0.0, '(2)2Φ': 0.0017, '(6)2Π': 0.0}",{2.5},2.5,(1)2Δ_5/2,(1)5/2,0.983
1,-119.860885,-4170.780000,0.000000,(1)2Δ,"{'(1)2Δ': 0.9829, '(1)2Σ+': 0.0, '(1)2Π': 0.0, '(1)4Δ': 0.0001, '(2)2Σ+': 0.0, '(1)4Π': 0.0004, '(2)2Δ': 0.001, '(2)2Π': 0.0, '(1)4Σ-': 0.0, '4Φ': 0.0046, '(3)2Σ+': 0.0, '4Σ+': 0.0, '(3)2Π': 0.0, '(2)4Π': 0.0052, '(1)2Φ': 0.0009, '(1)2Σ-': 0.0, '(3)2Δ': 0.0001, '(2)4Δ': 0.0, '(4)2Π': 0.0, '(2)4Σ-': 0.0, '(3)4Π': 0.0027, '(2)2Σ-': 0.0, '(4)2Δ': 0.0, '(4)2Σ+': 0.0, '(5)2Π': 0.0, '2Γ': 0.0, '(5)2Δ': 0.0002, '(5)2Σ+': 0.0, '(2)2Φ': 0.0017, '(6)2Π': 0.0}",{2.5},2.5,(1)2Δ_5/2,(2)5/2,0.994
2,-119.854930,-2863.760000,1307.020000,(1)2Σ+,"{'(1)2Δ': 0.0, '(1)2Σ+': 0.6869, '(1)2Π': 0.2971, '(1)4Δ': 0.0, '(2)2Σ+': 0.0001, '(1)4Π': 0.0096, '(2)2Δ': 0.0, '(2)2Π': 0.0006, '(1)4Σ-': 0.0015, '4Φ': 0.0, '(3)2Σ+': 0.001, '4Σ+': 0.0, '(3)2Π': 0.0008, '(2)4Π': 0.0, '(1)2Φ': 0.0, '(1)2Σ-': 0.001, '(3)2Δ': 0.0, '(2)4Δ': 0.0002, '(4)2Π': 0.0, '(2)4Σ-': 0.0, '(3)4Π': 0.0003, '(2)2Σ-': 0.0002, '(4)2Δ': 0.0, '(4)2Σ+': 0.0004, '(5)2Π': 0.0003, '2Γ': 0.0, '(5)2Δ': 0.0, '(5)2Σ+': 0.0, '(2)2Φ': 0.0, '(6)2Π': 0.0}",{0.5},0.5,(1)2Σ+_1/2,(1)1/2,0.982
3,-119.854930,-2863.760000,1307.020000,(1)2Σ+,"{'(1)2Δ': 0.0, '(1)2Σ+': 0.6869, '(1)2Π': 0.2971, '(1)4Δ': 0.0, '(2)2Σ+': 0.0001, '(1)4Π': 0.0096, '(2)2Δ': 0.0, '(2)2Π': 0.0006, '(1)4Σ-': 0.0015, '4Φ': 0.0, '(3)2Σ+': 0.001, '4Σ+': 0.0, '(3)2Π': 0.0008, '(2)4Π': 0.0, '(1)2Φ': 0.0, '(1)2Σ-': 0.001, '(3)2Δ': 0.0, '(2)4Δ': 0.0002, '(4)2Π': 0.0, '(2)4Σ-': 0.0, '(3)4Π': 0.0003, '(2)2Σ-': 0.0002, '(4)2Δ': 0.0, '(4)2Σ+': 0.0004, '(5)2Π': 0.0003, '2Γ': 0.0, '(5)2Δ': 0.0, '(5)2Σ+': 0.0, '(2)2Φ': 0.0, '(6)2Π': 0.0}",{0.5},0.5,(1)2Σ+_1/2,(2)1/2,0.893
4,-119.854576,-2785.940000,1384.840000,(1)2Π,"{'(1)2Δ': 0.2754, '(1)2Σ+': 0.0, '(1)2Π': 0.7067, '(1)4Δ': 0.0, '(2)2Σ+': 0.0, '(1)4Π': 0.0003, '(2)2Δ': 0.0002, '(2)2Π': 0.0, '(1)4Σ-': 0.0054, '4Φ': 0.0034, '(3)2Σ+': 0.0, '4Σ+': 0.0002, '(3)2Π': 0.003, '(2)4Π': 0.0011, '(1)2Φ': 0.0, '(1)2Σ-': 0.0, '(3)2Δ': 0.0006, '(2)4Δ': 0.0003, '(4)2Π': 0.0002, '(2)4Σ-': 0.0, '(3)4Π': 0.0009, '(2)2Σ-': 0.0, '(4)2Δ': 0.0004, '(4)2Σ+': 0.0, '(5)2Π': 0.0001, '2Γ': 0.0, '(5)2Δ': 0.0014, '(5)2Σ+': 0.0, '(2)2Φ': 0.0, '(6)2Π': 0.0003}",{1.5},1.5,(1)2Π_3/2,(1)3/2,0.952
5,-119.854576,-2785.940000,1384.840000,(1)2Π,"{'(1)2Δ': 0.2754, '(1)2Σ+': 0.0, '(1)2Π': 0.7067, '(1)4Δ': 0.0, '(2)2Σ+': 0.0, '(1)4Π': 0.0003, '(2)2Δ': 0.0002, '(2)2Π': 0.0, '(1)4Σ-': 0.0054, '4Φ': 0.0034, '(3)2Σ+': 0.0, '4Σ+': 0.0002, '(3)2Π': 0.003, '(2)4Π': 0.0011, '(1)2Φ': 0.0, '(1)2Σ-': 0.0, '(3)2Δ': 0.0006, '(2)4Δ': 0.0003, '(4)2Π': 0.0002, '(2)4Σ-': 0.0, '(3)4Π': 0.0009, '(2)2Σ-': 0.0, '(4)2Δ': 0.0004, '(4)2Σ+': 0.0, '(5)2Π': 0.0001, '2Γ': 0.0, '(5)2Δ': 0.0014, '(5)2Σ+': 0.0, '(2)2Φ': 0.0, '(6)2Π': 0.0003}",{1.5},1.5,(1)2Π_3/2,(2)3/2,0.862
6,-119.816552,5559.290000,9730.070000,(1)2Π,"{'(1)2Δ': 0.0, '(1)2Σ+': 0.2431, '(1)2Π': 0.5948, '(1)4Δ': 0.0039, '(2)2Σ+': 0.0732, '(1)4Π': 0.0508, '(2)2Δ': 0.0, '(2)2Π': 0.0001, '(1)4Σ-': 0.001, '4Φ': 0.0, '(3)2Σ+': 0.0051, '4Σ+': 0.0002, '(3)2Π': 0.0021, '(2)4Π': 0.0046, '(1)2Φ': 0.0, '(1)2Σ-': 0.0122, '(3)2Δ': 0.0, '(2)4Δ': 0.003, '(4)2Π': 0.0002, '(2)4Σ-': 0.0005, '(3)4Π': 0.0018, '(2)2Σ-': 0.0005, '(4)2Δ': 0.0, '(4)2Σ+': 0.0015, '(5)2Π': 0.0005, '2Γ': 0.0, '(5)2Δ': 0.0, '(5)2Σ+': 0.0001, '(2)2Φ': 0.0, '(6)2Π': 0.0011}",{0.5},0.5,(1)2Π_1/2,(3)1/2,0.858
7,-119.816552,5559.290000,9730.070000,(1)2Π,"{'(1)2Δ': 0.0, '(1)2Σ+': 0.2431, '(1)2Π': 0.5948, '(1)4Δ': 0.0039, '(2)2Σ+': 0.0732

Total of CCwt column = 13.000
The CC terms are ['(1)2Σ+', '(1)2Δ', '(1)2Π', '(1)4Δ', '(1)4Π']
